In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [2]:
import pandas as pd
pd.options.display.max_rows = 3000
pd.options.display.max_columns = 100
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 텍스트 전처리
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from konlpy.tag import Okt 
import MeCab
mecab = MeCab.Tagger()
import re 
from string import punctuation
import requests
import pickle
import ast

# 토픽모델링
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
import pyLDAvis 
import pyLDAvis.gensim

C:\ProgramData\Anaconda3\envs\textmining\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


#### 미리 만들어둔 리뷰데이터 불러오기

In [3]:
df = pd.read_csv("D:/review.csv")

In [4]:
df

,DATE,STAR,REVIEW,LIKE
0,2019-12-27,5,부산에도 기다리던 지역화폐가 나왔네요 앱도 깔끔하고 카드도 바로 신청하고 잘 사용하...,8
1,2019-12-27,5,화이팅,4
2,2019-12-28,5,기다렸었는데 앞으로 잘 쓸게요^^,4
3,2019-12-28,5,동백전으로 여기저기 많이 사용해 볼게요. 부산 경제에도 많은 도움이 되면 좋겠습니다.,4
4,2019-12-28,5,부산 동백전 쓰기 편하게 잘 만들어졌네요 부산에서 혜택이 많았겠네요,5
5,2019-12-28,5,잘 사용해 볼게요. 동백전으로 혜택 많이 받으면 좋겠습니다.,4
6,2019-12-28,5,가자 글로벌 테스트베드 부산!,3
7,2019-12-29,5,오~ 부산도 드디어 모바일 지역화폐가 나왔네요!~ 앱설치하고 카드신청도 해봤는데요 ...,53
8,2019-12-30,5,드디어 기다리던 동백전앱이 출시됐네요 앞으로 이용 많이 하겠습니다,2
9,2019-12-30,4,군더더기 없는 ui가 마음에 드는군요. 많은 사업장에서 사용가능하면 더 좋겠습니다.,1


In [5]:
df.shape

(2871, 4)

#### corpus (말뭉치) 생성

In [6]:
data_positive = df[df['STAR'] > 3] # 긍정 리뷰
data_negative = df[df['STAR'] < 3] # 부정 리뷰

In [7]:
corpus_posi = data_positive['REVIEW'] # 긍정리뷰
corpus_nega = data_negative['REVIEW'] # 부정리뷰

#### 텍스트 전처리
- 자음모음만으로 구성된 것 제거
- 특수문자 제거
- 숫자 제거

In [8]:
def message_cleaning(docs):

  
   
    # Series의 object를 str로 변경.
    docs = [str(doc) for doc in docs]
    
    
    # 1. 자음 모음 제거하기
    pattern1 = re.compile("[ㄱ-ㅎ]*[ㅏ-ㅢ]*")
    docs = [pattern1.sub("", doc) for doc in docs]
    # 2. 특수문자 제거
    pattern2 = re.compile("[\{\}\[\]\/?.,;:'|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]")
    docs = [pattern2.sub("", doc) for doc in docs]
    
    
    return docs


<>:13: DeprecationWarning: invalid escape sequence \{
<>:13: DeprecationWarning: invalid escape sequence \{
<>:13: DeprecationWarning: invalid escape sequence \{
<ipython-input-8-f6a8cd553c2c>:13: DeprecationWarning: invalid escape sequence \{
  pattern2 = re.compile("[\{\}\[\]\/?.,;:'|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]")


#### 명사추출

In [9]:
import re
import MeCab # 형태소 분석기
mecab = MeCab.Tagger()
import requests
import pickle
import ast
## mecab형태소 분석기로 명사 추출하는 함수
def mecab_nouns(text): 
    nouns = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+")
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]] 
    
    # 추출한 token중에 POS가 명사 분류에 속하는 토큰만 선택.
    for token in temp:
        #동사(어근)까지 추출할려면 "VV"까지
        if token[1] == "NNG" or token[1] == "NNP" or token[1] == "NNB" or token[1] == "NNBC" or token[1] == "NP" or token[1] == "NR"or token[1] == "NNS"or token[1] == "NP" or token[1] == "NR"or token[1] == "NNS" or token[1]== "SL" :
            nouns.append(token[0])
    return nouns

In [10]:
## 텍스트 정제
cleaned_corpus_posi = message_cleaning(corpus_posi)
cleaned_corpus_nega = message_cleaning(corpus_nega)

#### 불용어 처리 및 한글자 제거

In [11]:

def define_stopwords(path):
    
    SW = set()
    #불용어를 추가하는 방법 1.
    #SW.add("동백전")
    
    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path) as f:
        for word in f:
            SW.add(word[:-1])
            
    return SW

from tqdm import tqdm_notebook # 시간 바

# 명사 추출한 것 중 SW에 포함되지 않으면서 한글자 제거
def text_tokenizing(corpus):   
    token_corpus = []
    # tqdm을 사용하여 진행 과정을 보기
    for n in tqdm_notebook(range(len(corpus))):
        token_text = mecab_nouns(corpus[n]) # 위에서 정의한 명사추출 함수 실행
        token_text = [word for word in token_text if word not in SW and len(word) >1]
        token_corpus.append(token_text)
    return token_corpus

SW = define_stopwords("D:/연구알바/stopwords-ko.txt")
SW

{'',
 'Good',
 '가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '건가',
 '건가요',
 '건지',
 '겁니까',
 '것',
 '것과 같이',
 '것들',
 '게다가',
 '게요',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '경우',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '공무원',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '굿',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너무',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '누가 알겠는가',
 '누구',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '

#### 사용자사전(분리되면 안되는 단어들을 따로 사전에 정의)
> 동백전
하나은행
부산은행
캐쉬백
캐시백
지역화폐
해결방안
고객센터
교통카드
체크카드
불편함
본인인증
본인 인증
인증번호
코나아이
삼성페이
오프라인
온라인
비번
비밀번호
아이디
전화번호
폰 번호
생년월일
홈페이지
소상공인
지원금
로코
큐알코드 등등

In [12]:
tokenized_text_posi = text_tokenizing(cleaned_corpus_posi)
print(tokenized_text_posi)
tokenized_text_nega = text_tokenizing(cleaned_corpus_nega)

C:\ProgramData\Anaconda3\envs\textmining\lib\site-packages\ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1304 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\envs\textmining\lib\site-packages\ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


[['지역화폐', '카드', '신청', '사용'], ['화이팅'], [], ['여기저기', '사용', '경제', '도움'], ['혜택'], ['사용', '혜택'], ['글로벌', '테스트', '베드'], ['모바일', '지역화폐', '설치', '카드', '신청', '구성', '수도', '명절', '카드', '부모', '지역', '페이', '수당', '나중', '복지', '비용', '이걸로', '부모', '기대'], ['출시', '이용'], ['군더더기', 'ui', '마음', '사업장', '사용', '가능'], [], ['이용', '시민', '성장'], ['대표', '이름', '이름', '이용', '혜택', '기대', '사용', '편리'], ['설치'], ['군더더기', '사용'], ['사용', '사용'], ['회원가입', '사람', '가입'], ['적립', '한도', '충전', '적립', '한도'], ['가맹점', '사용', '캐시백', '만족', '가맹점', '가맹점', '전체', '가맹점', '사용', '가능', '가게', '검색', '관련', '가게', '아무것', '평소', '이용', '가게', '사용', '불편', '동네', '사용', '가능', '유무', '불편', '설정', '설정', '활성', '문제', '업데이트', '감사'], ['네트워크', '네트워크', '실행', '수정'], ['교통카드', '이용', '내역', '카드', '이용', '내역', '교통카드', '금액'], [], ['오류', '캐시백', '재미', '기간', '단위', '이용자', '캐시백', '혜택', '전체', '총합', '한눈', '직관', '만족도'], ['가맹점', '기능', '개선', '가맹점', '기능', '위치', '지도', '사용', '가능', '가맹점', '검색', '불편'], ['단위', '충전'], ['택시', '결제'], ['설치', '금일', '저녁', '카드', '주니', '포스', '결제', '플로', '결제', '확인', '삭제', '에러',

  0%|          | 0/1349 [00:00<?, ?it/s]

#### 문서 단어 행렬

In [13]:
def dic_and_bow(clean_text):
    
    # 데이터를 dictionary 형태로 명사 list 만들기 
    dictionary = corpora.Dictionary(clean_text) 
    
    # 출현빈도가 너무 적은 단어는 제거 
    dictionary.filter_extremes(no_below=5) 
    
    # 명사 형태로 말뭉치 만들기 
    corpus = [dictionary.doc2bow(text) for text in clean_text]
    
    # TF-IDF으로 변환 
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    corpus = corpus_tfidf 
    
    return corpus ,dictionary

corpus_posi ,dictionary_posi = dic_and_bow(tokenized_text_posi)
corpus_nega ,dictionary_nega = dic_and_bow(tokenized_text_nega)
corpus_posi[0]


[(0, 0.2401471933668554),
 (1, 0.6342873726171315),
 (2, 0.6460593140855595),
 (3, 0.3501659851267321)]

- TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단합니다. 
- TF-IDF 값이 낮으면 중요도가 낮은 것이며, TF-IDF 값이 크면 중요도가 큰 것입니다. 
- 즉, the나 a와 같이 불용어의 경우에는 모든 문서에 자주 등장하기 마련이기 때문에 자연스럽게 불용어의 TF-IDF의 값은 다른 단어의 TF-IDF에 비해서 낮아지게 됩니다.
> 모든 문서에서 자주 등장하는 단어의 중요도를 낮게 평가 (?!)

- TF-IDF를 돌린 후 명사 추출 > corpus에서 문장에서 단어의 중요도를 평가해서 추출한 것을 다시 명사로 추출하면 그 중요도가 의미가 없어짐(중요하다는 단어가 동사일 경우 중요함에도 사라지기에)

- 명사 추출 후 TF-IDF를 돌리기 > 모든 문서에서 자주 등장하는 명사가 중요하지 않다고 판단하여 핵심 명사들이 사라지는 경우가 생김

- 단순 빈도로 인한 명사 추출뿐인가..

#### 최적 Topic 개수 산출

<span style="color:red">coherence 보단 perplexity를 우선적으로 보는 게 좋음</span>


__(1) Coherence Model__

Topic이 얼마나 의미론적으로 일관성 있는지 판단.
수치가 높을수록 일관성 높음. 0.55 정도면 우수하다고 함.
Coherence가 너무 높아지면 정보의 양이 줄어들게 되고, coherence가 너무 낮아 정보들이 인관성이 없다면 분석의 의미가 낮아지게 됨.

.3 is bad

.4 is low

.55 is okay

.65 might be as good as it is going to get

.7 is nice

.8 is unlikely and

.9 is probably wrong

- 매번 모델을 돌릴때마다 토픽이 달라지기에 seed 고정이 필요할 것 같다

In [14]:
# coherence_values = []

# for i in tqdm_notebook(range(2,30)) : 
#     ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word = dictionary) # 파라미터는 기본으로 설정 
#     coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized_text, dictionary=dictionary, topn=10, coherence='c_v')
#     coherence_lda = coherence_model_lda.get_coherence()
#     coherence_values.append(coherence_lda)

In [15]:
# x = range(2,30)
# plt.plot(x, coherence_values)
# plt.xlabel("Number of Topics")
# plt.ylabel("Coherence Score")
# plt.show()

__(2) 언어 모델 평가방법 (PPL: Perplexity)__

확률 모델이 결과를 얼마나 정확하게 예측하는지 나타내는 값.
동일 모델 내 파라미터에 따른 성능 평가할 때 사용.
선정된 토픽 개수마다 학습시켜 가장 낮은 값을 보이는 구간을 찾아 최적화된 토픽의 개수 선정. 
수치가 낮을수록 높은 정확도.
Coherence Score와 함께 고려해야함.

<span style="color:red">train, test set 나누기</span>

- 8:2로 나누는 게 좋다고 함. 
- 80%로 학습한 lda model을 20%의 테스트셋에 적용시켜 수치를 보고, 가장 좋은 토픽의 개수를 찾아 전체 데이터셋에 토픽모델링 적용

In [16]:
# len(corpus)

In [17]:
# corpus_train = corpus[:2200]
# corpus_test = corpus[2200:]

In [18]:
# perplexity_values=[]

# for i in tqdm_notebook(range(2,100)): 
#     ldamodel = gensim.models.ldamodel.LdaModel(corpus_train, 
#                                                num_topics=i, 
#                                                id2word=dictionary, 
#                                                alpha="auto", eta="auto")

#     perplexity_values.append(ldamodel.log_perplexity(corpus_test))

In [19]:
# x = range(2,100)
# plt.plot(x, perplexity_values)
# plt.xlabel("Number of Topics")
# plt.ylabel("Perplexity Score")
# plt.show()

#### 긍정리뷰 토픽모델링 실행

In [20]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 8

NUM_TOPIC_WORDS = 10

def print_topic_words(model):

    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

# LDA를 실행.
model_posi = models.ldamodel.LdaModel(corpus_posi, num_topics=NUM_TOPICS, id2word=dictionary_posi, alpha=0.1,iterations=1000,random_state=100,passes=100)
model_nega= models.ldamodel.LdaModel(corpus_nega, num_topics=NUM_TOPICS, id2word=dictionary_nega,  alpha=0.1,iterations=1000,random_state=100,passes=100)

print_topic_words(model_posi)


Printing topic words.

Topic ID: 0
	카드	0.1540868729352951
	기대	0.10027919709682465
	캐쉬백	0.07898606359958649
	오류	0.04843571037054062
	발급	0.047818634659051895
	가능	0.046893879771232605
	이용	0.045359767973423004
	경제	0.045340247452259064
	기존	0.030531900003552437
	개꿀	0.030419304966926575


Topic ID: 1
	편리	0.37287962436676025
	최고	0.27094823122024536
	사용	0.07075205445289612
	페이	0.03947552293539047
	결재	0.03193792700767517
	부족	0.026626920327544212
	재미	0.019661683589220047
	정보	0.013984307646751404
	인천	0.012911660596728325
	잔액	0.010480184108018875


Topic ID: 2
	사용	0.3977852165699005
	포인트	0.16012664139270782
	할인	0.08199762552976608
	화이팅	0.04411805793642998
	처음	0.03185059875249863
	검색	0.030323417857289314
	이득	0.02616628259420395
	방법	0.02231799066066742
	가능	0.018468394875526428
	직관	0.016446107998490334


Topic ID: 3
	감사	0.21449896693229675
	적립	0.1705041527748108
	서비스	0.05922529846429825
	지역화폐	0.052748892456293106
	대박	0.03185388818383217
	부탁	0.030844079330563545
	사용	0.03044491820037365
	인천	0.029893258

In [21]:
word_dict1 = {};

for i in range(NUM_TOPICS):

    words = model_posi.show_topic(i)

    word_dict1['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]

pd.DataFrame(word_dict1)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08
0,카드,편리,사용,감사,혜택,충전,유용,만족
1,기대,최고,포인트,적립,캐시백,금액,가맹점,10프로
2,캐쉬백,사용,할인,서비스,결제,필요,도움,업데이트
3,오류,페이,화이팅,지역화폐,설치,기능,지역,불편
4,발급,결재,처음,대박,연장,사용,사람,강추
5,가능,부족,검색,부탁,사랑,개선,접속,계좌
6,이용,재미,이득,사용,에러,내역,확인,카드
7,경제,정보,방법,인천,사용,기간,사용,등록
8,기존,인천,가능,택시,잔액,아이폰,생각,삼성페이
9,개꿀,잔액,직관,디자인,실행,발전,제도,사용


긍정 리뷰 토픽 네이밍
1. 기대 2. 편리한 결제 3. 사용가능한 가맹점 4. 충전 및 적립 혜택 5. 혜택 편리 6. 사용 편리 7. 감사 도움 8. 사용 추천 9. 카드 유용 10. 포인트 및 할인 11. 사용 감사 12. 사용 만족 13.포인트 10프로 14. 추천 15.캐시백 유용 16.캐시백 감사 17. 지역 18. 사용 경제 19. 감사 20. 편리,유용 21 최고 22. 카드 사용 23.캐시백 혜택 24.할인 혜택 25. 도움 감사 26.캐시백 가맹점 27. 사용 감사 28.사용 최고 29. 적립 10프로 30.만족 최고 

토픽 네이밍 그룹화 
1. 앱 만족(만족,기대,추천) - 1,7,11,12,25,27,28,30
2. 사용편리 - 2,6,20,22
3. 결제 및 충전- 21
4. 지역,경제 - 17,18,19
5. 캐시백 및 혜택,적립 4,5,10,13,15,16,23,24,29
6. 가맹점 - 3,26
7. 오류 - 9,14

In [22]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행.
data1 = pyLDAvis.gensim.prepare(model_posi, corpus_posi, dictionary_posi)
data1

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.119939  0.139329       1        1  13.857900
0      0.150815  0.128218       2        1  13.811586
5      0.056218 -0.039506       3        1  13.604169
7      0.103270 -0.074267       4        1  13.126464
3     -0.011919  0.050919       5        1  11.705395
2     -0.145632  0.061382       6        1  11.587713
4      0.025609 -0.296323       7        1  11.178063
1     -0.298300  0.030249       8        1  11.128710, topic_info=      Term       Freq      Total Category  logprob  loglift
17      편리  46.000000  46.000000  Default  30.0000  30.0000
63      최고  33.000000  33.000000  Default  29.0000  29.0000
0       사용  89.000000  89.000000  Default  28.0000  28.0000
7       혜택  28.000000  28.000000  Default  27.0000  27.0000
25      감사  28.000000  28.000000  Default  26.0000  26.0000
33     캐시백  33.000000  33.000000  Default  25.0000  25.0000
21      적립  23.000000  23.000000  Default  24.0000  24.0000
52     포인트  21.000000  21.000000  Default  23.0000  23.0000
83      유용  19.000000  19.000000  Default  22.0000  22.0000
3       카드  36.000000  36.000000  Default  21.0000  21.0000
8       기대  16.000000  16.000000  Default  20.0000  20.0000
28      만족  19.000000  19.000000  Default  19.0000  19.0000
22      충전  20.000000  20.000000  Default  18.0000  18.0000
24     가맹점  17.000000  17.000000  Default  17.0000  17.0000
6       도움  14.000000  14.000000  Default  16.0000  16.0000
100   10프로  15.000000  15.000000  Default  15.0000  15.0000
32    업데이트  12.000000  12.000000  Default  14.0000  14.0000
66      할인  11.000000  11.000000  Default  13.0000  13.0000
64     캐쉬백  14.000000  14.000000  Default  12.0000  12.0000
37      금액  11.000000  11.000000  Default  11.0000  11.0000
10      지역  13.000000  13.000000  Default  10.0000  10.0000
54      필요  10.000000  10.000000  Default   9.0000   9.0000
77      강추   9.000000   9.000000  Default   8.0000   8.0000
82     서비스   8.000000   8.000000  Default   7.0000   7.0000
30      불편  13.000000  13.000000  Default   6.0000   6.0000
19      사람   9.000000   9.000000  Default   5.0000   5.0000
44      기능  10.000000  10.000000  Default   4.0000   4.0000
2     지역화폐   7.000000   7.000000  Default   3.0000   3.0000
5       경제  11.000000  11.000000  Default   2.0000   2.0000
69      접속   7.000000   7.000000  Default   1.0000   1.0000
83      유용  18.790340  19.577815   Topic1  -2.0926   1.9353
6       도움  13.581998  14.369528   Topic1  -2.4172   1.9200
69      접속   7.211822   7.999403   Topic1  -3.0502   1.8727
19      사람   8.507965   9.671485   Topic1  -2.8849   1.8481
24     가맹점  15.342543  17.592670   Topic1  -2.2953   1.8395
97      제도   3.903386   4.690841   Topic1  -3.6641   1.7925
10      지역  11.014336  13.845167   Topic1  -2.6267   1.7476
23      가게   3.044803   3.832287   Topic1  -3.9125   1.7463
121     화폐   2.947706   3.735307   Topic1  -3.9449   1.7395
55      활용   2.908071   3.695815   Topic1  -3.9585   1.7366
87     적립금   3.883703   5.712165   Topic1  -3.6692   1.5905
85      부분   2.476014   3.783067   Topic1  -4.1193   1.5524
110     이전   3.288935   5.070971   Topic1  -3.8354   1.5433
18      가입   2.864356   4.472933   Topic1  -3.9736   1.5306
49      확인   6.325309   9.943667   Topic1  -3.1814   1.5239
65      생각   5.241234   8.356317   Topic1  -3.3694   1.5099
86      리뷰   1.463473   2.812669   Topic1  -4.6451   1.3230
56      소비   1.404609   3.894715   Topic1  -4.6862   0.9565
71      사항   1.342465   3.816340   Topic1  -4.7314   0.9315
5       경제   3.780658  11.341016   Topic1  -3.6960   0.8778
15      시민   1.920909   6.832068   Topic1  -4.3731   0.7075
36      실행   1.611678   5.943720   Topic1  -4.5487   0.6713
88      추천   1.774971   6.642674   Topic1  -4.4522   0.6566
20    회원가입   0.831830   3.225635   Topic1  -5.2101   0.6211
104     잔액   2.438598   9.865267   Topic1  -4.1345   0.5787
30      불편   3.037702  13.493668   Topic1  -3.9148   0.48

#### 부정리뷰 토픽모델링 실행

In [23]:
word_dict2 = {};

for i in range(NUM_TOPICS):

    words = model_nega.show_topic(i)

    word_dict2['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]

pd.DataFrame(word_dict2)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08
0,충전,불편,카드,가맹점,업데이트,접속,오류,환불
1,설치,금액,발급,예전,내역,다운,실행,짜증
2,연결,사용,기존,기능,사용,최악,네트워크,에러
3,전화,캐시백,신청,인증,이용,업데이트,결제,로그인
4,계좌,이전,등록,사용,삭제,하루,접속,이해
5,고객센터,확인,삼성페이,검색,캐시백,종일,데이터,부족
6,문제,잔액,신규,문자,직관,방법,와이파이,서비스
7,포인트,캐시,쓰레기,가능,개선,며칠,불가,운영
8,인증번호,개선,사용,불편,적립,부탁,관리,회사
9,불편함,캐쉬백,선불카드,가입,충전,대기,서버,발급


부정리뷰 토픽 네이밍
1. 연결 설치 2. 불편 삼성페이 3. 카드 발급 4. 사용 불편 5. 환불 6. 접속 7. 오류 8. 로그인 9. 업데이트 10. 업데이트 11.업데이트 12 접속 13.카드 등록 14.결제 금액 15. 신규 카드 발급 16. 기존 이전 17 포인트 사용 불편 18.접속 19.충전 금액 20.충전 내역 21. 업데이트 22 잔액 23. 카드신청 24. 카드 결제 25. 업데이트 26 예전 사용 27. 이관 29. 사용 불편 30. 오류 

토픽 네이밍 그룹화
1. 연결,접속 - 1,5,6,18
2. 업데이트 - 9,10,11,25
2. 카드 발급 및 등록 - 3 ,13,15,23,24
3. 사용 불편 - 4,17,29
3. 충전 및 금액 내역 - 12,19,20,22
4. 기존,이전 및 이관 - 8,16,26
5. 결제,삼성페이 - 2,14,21,27
6. 설치 - 28
7. 네트워크,서버 -7

In [24]:
# pyLDAvis 실행.
data2 = pyLDAvis.gensim.prepare(model_nega, corpus_nega, dictionary_nega)
data2

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.171781 -0.137945       1        1  22.520027
0     -0.096832 -0.057289       2        1  14.132231
1      0.047802  0.176668       3        1  13.847417
6     -0.245778 -0.057829       4        1  11.635146
4     -0.007486  0.158718       5        1  11.102552
3      0.133430  0.072606       6        1   9.828909
7      0.111119 -0.136246       7        1   8.761857
5     -0.114037 -0.018684       8        1   8.171861, topic_info=      Term       Freq      Total Category  logprob  loglift
66    업데이트  50.000000  50.000000  Default  30.0000  30.0000
44      접속  44.000000  44.000000  Default  29.0000  29.0000
17      오류  40.000000  40.000000  Default  28.0000  28.0000
35     가맹점  28.000000  28.000000  Default  27.0000  27.0000
79      실행  27.000000  27.000000  Default  26.0000  26.0000
23      불편  59.000000  59.000000  Default  25.0000  25.0000
93      환불  16.000000  16.000000  Default  24.0000  24.0000
1     네트워크  23.000000  23.000000  Default  23.0000  23.0000
29      발급  44.000000  44.000000  Default  22.0000  22.0000
162     연결  25.000000  25.000000  Default  21.0000  21.0000
9       카드  75.000000  75.000000  Default  20.0000  20.0000
28      금액  29.000000  29.000000  Default  19.0000  19.0000
27      설치  26.000000  26.000000  Default  18.0000  18.0000
65      내역  19.000000  19.000000  Default  17.0000  17.0000
61      다운  11.000000  11.000000  Default  16.0000  16.0000
69      기존  34.000000  34.000000  Default  15.0000  15.0000
8       신청  22.000000  22.000000  Default  14.0000  14.0000
6       결제  36.000000  36.000000  Default  13.0000  13.0000
59      최악  19.000000  19.000000  Default  12.0000  12.0000
289     예전  16.000000  16.000000  Default  11.0000  11.0000
58      전화  14.000000  14.000000  Default  10.0000  10.0000
25     캐시백  24.000000  24.000000  Default   9.0000   9.0000
192     기능  17.000000  17.000000  Default   8.0000   8.0000
57      에러   9.000000   9.000000  Default   7.0000   7.0000
60    고객센터  12.000000  12.000000  Default   6.0000   6.0000
90      계좌  14.000000  14.000000  Default   5.0000   5.0000
134     짜증  24.000000  24.000000  Default   4.0000   4.0000
273     신규  17.000000  17.000000  Default   3.0000   3.0000
18    와이파이  10.000000  10.000000  Default   2.0000   2.0000
238     이전  28.000000  28.000000  Default   1.0000   1.0000
273     신규  16.410015  17.211399   Topic1  -3.6117   1.4431
8       신청  20.938116  22.161714   Topic1  -3.3680   1.4340
254     연동   8.358571   9.159733   Topic1  -4.2863   1.3992
265   선불카드  12.830091  14.471931   Topic1  -3.8578   1.3703
56     쓰레기  15.481511  17.466466   Topic1  -3.6700   1.3701
262     이관  12.699281  14.385428   Topic1  -3.8681   1.3661
170     낭비   5.417665   6.218794   Topic1  -4.7200   1.3529
205   플라스틱   5.225340   6.026371   Topic1  -4.7561   1.3481
119     강제   8.747073  10.213281   Topic1  -4.2409   1.3358
199     세금   4.565924   5.367100   Topic1  -4.8910   1.3291
267     공제   3.995570   4.796828   Topic1  -5.0244   1.3080
269     소득   3.995570   4.796828   Topic1  -5.0244   1.3080
69      기존  28.287753  34.312233   Topic1  -3.0672   1.2977
29      발급  36.427097  44.298922   Topic1  -2.8143   1.2951
280     환경   3.607623   4.408601   Topic1  -5.1266   1.2903
185    수수료   3.997407   4.902649   Topic1  -5.0240   1.2866
270     유도   3.524687   4.325784   Topic1  -5.1498   1.2860
242     실물   6.902668   8.563058   Topic1  -4.4777   1.2752
271  인터페이스   6.518215   8.226727   Topic1  -4.5350   1.2580
107   체크카드   7.244483   9.145754   Topic1  -4.4294   1.2577
68      국민   2.735744   3.539833   Topic1  -5.4032   1.2331
219     비용   2.609922   3.410976   Topic1  -5.4503   1.2231
283     의무   2.574124   3.375156   Topic1  -5.4641   1.2198
102     동의   3.400354   4.602846   Topic1  -5.1857   1.1880
91      등록  18.451315  25.099827   Topic1  -3.4945   1.1830
196     거기   2.145076   2.946518   Topic1  -5.6464   1.17